In [1]:
import requests
import json
import os
import pandas as pd
import numpy as np
from pyproj import Proj
from shapely.geometry import shape
import pickle

In [2]:
headers = {"token": "kzleSzqsKtoPFsCbPuoPWOhXvXZMfTLp"}
#headers = {"token": "xVEIkLnfHyheHhvoheZSxesUerlyrxGN"}

In [3]:
all_project_stations = pd.read_csv('./data/all_project_stations.csv')
all_project_stations

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
0,0.9995,304.8,METERS,GHCND_USW00013969,36.73667,-97.10194,10/16/2018,1/1/1948,"PONCA CITY MUNICIPAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.262218,Thunder Ranch
1,0.9872,299.9,METERS,GHCND_USW00003965,36.16240,-97.08940,10/16/2018,11/1/1996,"STILLWATER REGIONAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.463805,Thunder Ranch
2,0.9675,272.8,METERS,GHCND_USC00348501,36.11750,-97.09500,10/17/2018,1893-01-01,"STILLWATER 2 W, OK US",2.139864,14,-97.306297,36.572365,0.501546,Thunder Ranch
3,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,10/16/2018,1/1/1948,"SIDNEY MUNICIPAL AIRPORT, NE US",1.783220,13,-103.322228,40.961955,0.362920,Peetz Table
4,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,9/30/2018,12/1/1908,"SIDNEY 6 NNW, NE US",1.783220,13,-103.322228,40.961955,0.402522,Peetz Table
5,0.9988,1130.5,METERS,GHCND_USC00051996,40.86000,-102.80300,10/17/2018,3/1/1996,"CROOK, CO US",1.783220,13,-103.322228,40.961955,0.529143,Peetz Table
6,0.9804,545.6,METERS,GHCND_USW00013962,32.41050,-99.68220,10/16/2018,8/1/1946,"ABILENE REGIONAL AIRPORT, TX US",2.567837,15,-100.173964,32.253115,0.516336,Horse Hollow
7,0.9411,594.1,METERS,GHCND_USC00414278,31.84555,-99.56068,10/17/2018,7/1/1953,"HORDS CREEK DAM, TX US",2.567837,15,-100.173964,32.253115,0.736361,Horse Hollow
8,0.8222,598.6,METERS,GHCND_USC00416499,31.45389,-100.49333,1/31/2008,12/1/1975,"O C FISHER DAM, TX US",2.567837,15,-100.173964,32.253115,0.860671,Horse Hollow
9,0.8793,786.4,METERS,GHCND_USW00024012,46.79940,-102.79720,10/16/2018,11/14/1938,"DICKINSON THEODORE ROOSEVELT REGIONAL AIRPORT,...",3.081404,16,-102.756389,46.656348,0.148759,Brady Wind I


In [4]:
len(all_project_stations)

1248

In [5]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?datacategoryid=TEMP&limit=56&datasetid=GHCND"
response = requests.get(url=url, headers=headers)
noaa_temp_data_types_json = response.json()
noaa_temp_data_types_df = pd.DataFrame(noaa_temp_data_types_json['results'])
temp_data_types = noaa_temp_data_types_df.id

In [6]:
noaa_temp_data_types_df

,datacoverage,id,maxdate,mindate,name
0,1,DATN,2018-03-03,1863-05-04,Number of days included in the multiday minimu...
1,1,DATX,2018-03-02,1863-05-04,Number of days included in the multiday maximu...
2,1,MDTN,2018-03-03,1863-05-04,Multiday minimum temperature (use with DATN)
3,1,MDTX,2018-03-02,1863-05-04,Multiday maximum temperature (use with DATX)
4,1,TAVG,2018-10-30,1874-10-13,Average Temperature.
5,1,TMAX,2018-10-30,1763-01-01,Maximum temperature
6,1,TMIN,2018-10-30,1763-01-01,Minimum temperature
7,1,TOBS,2018-10-30,1876-11-27,Temperature at the time of observation


In [7]:
temp_data_types

0    DATN
1    DATX
2    MDTN
3    MDTX
4    TAVG
5    TMAX
6    TMIN
7    TOBS
Name: id, dtype: object

In [8]:
temp_data_types = temp_data_types[4:7]

In [9]:
temp_data_types

4    TAVG
5    TMAX
6    TMIN
Name: id, dtype: object

In [10]:
#parameters for weather query
datasetid='GHCND'
datatypeid="AWND"
stationid='GHCND:USW00013969'
units='standard'
startdate='2016-01-01'
enddate='2016-12-31'
limit = 366

In [11]:
def get_temp_data_daily_summary(stationid, datatypeid, units = 'standard', startdate = '2016-01-01', enddate='2016-12-31', limit =1000, offset = 25):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=" +\
    datatypeid + "&stationid=" + stationid + "&units=" + units + "&startdate=" + startdate +\
    "&enddate=" + enddate + "&limit=" + str(limit)
    response = requests.get(url=url, headers=headers)
    station_temp_report_json = response.json()
    station_temp_report_df = pd.DataFrame(station_temp_report_json['results'])
    return(station_temp_report_df)

In [12]:
station_count = 0
query_count = 0
start_date = "2016-01-01"
end_date = "2016-12-31"
for index, row in all_project_stations.iterrows():
    stationid = row['id'].replace("_", ":")
    stationid_dir_name = row["id"]
    projectid = row["p_name"].replace("/","&")
    failed_datatypes = []
    station_count += 1
    for w in temp_data_types:
        try:
            query_count += 1
            print("Project:", projectid, " | Station:", stationid, " | Feature:", w,
                  " | Station Count:", station_count, " | Query Count:", query_count)
            downloaded_data = get_temp_data_daily_summary(stationid, w, units = 'standard',
                                                          startdate=start_date , enddate=end_date, limit=1000)
            
            
            outname = w + "_" + start_date + "_" + end_date + ".csv"
            outdir = './data_2016/weather_2016/projects/' + projectid + "/" + stationid_dir_name
            if not os.path.exists(outdir):
                os.makedirs(outdir)
            fullname = os.path.join(outdir, outname)    
            downloaded_data.to_csv(fullname)
            
            #file_path = './data/weather_2016/projects/' + projectid + "/" + stationid_dir_name + '/' + w + \
            #"_" + start_date + "_" + end_date + ".csv"
            downloaded_data.to_csv(file_path, index=False)
            
        except:
            failed_datatypes.append(w)
    
    outdir = './data_2016/weather_2016/projects/' + projectid + "/" + stationid_dir_name
    outname = 'failed_datatypes'
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    fullname = os.path.join(outdir, outname) 
    with open(fullname , 'w') as f:
        for item in failed_datatypes:
            f.write("%s\n" % item)

Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: TAVG  | Station Count: 1  | Query Count: 1
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: TMAX  | Station Count: 1  | Query Count: 2
Project: Thunder Ranch  | Station: GHCND:USW00013969  | Feature: TMIN  | Station Count: 1  | Query Count: 3
Project: Thunder Ranch  | Station: GHCND:USW00003965  | Feature: TAVG  | Station Count: 2  | Query Count: 4
Project: Thunder Ranch  | Station: GHCND:USW00003965  | Feature: TMAX  | Station Count: 2  | Query Count: 5
Project: Thunder Ranch  | Station: GHCND:USW00003965  | Feature: TMIN  | Station Count: 2  | Query Count: 6
Project: Thunder Ranch  | Station: GHCND:USC00348501  | Feature: TAVG  | Station Count: 3  | Query Count: 7
Project: Thunder Ranch  | Station: GHCND:USC00348501  | Feature: TMAX  | Station Count: 3  | Query Count: 8
Project: Thunder Ranch  | Station: GHCND:USC00348501  | Feature: TMIN  | Station Count: 3  | Query Count: 9
Project: Peetz Table  | Stat

Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00013980  | Feature: TMIN  | Station Count: 25  | Query Count: 75
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: TAVG  | Station Count: 26  | Query Count: 76
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: TMAX  | Station Count: 26  | Query Count: 77
Project: Flat Ridge 2 Expansion  | Station: GHCND:USW00003957  | Feature: TMIN  | Station Count: 26  | Query Count: 78
Project: Flat Ridge 2 Expansion  | Station: GHCND:USC00343740  | Feature: TAVG  | Station Count: 27  | Query Count: 79
Project: Flat Ridge 2 Expansion  | Station: GHCND:USC00343740  | Feature: TMAX  | Station Count: 27  | Query Count: 80
Project: Flat Ridge 2 Expansion  | Station: GHCND:USC00343740  | Feature: TMIN  | Station Count: 27  | Query Count: 81
Project: Chisholm View  | Station: GHCND:USC00343740  | Feature: TAVG  | Station Count: 28  | Query Count: 82
Project: Chisholm View  | Station: GHCND:USC00343740  | F

Project: California Ridge  | Station: GHCND:USC00116344  | Feature: TMIN  | Station Count: 49  | Query Count: 147
Project: California Ridge  | Station: GHCND:USW00014806  | Feature: TAVG  | Station Count: 50  | Query Count: 148
Project: California Ridge  | Station: GHCND:USW00014806  | Feature: TMAX  | Station Count: 50  | Query Count: 149
Project: California Ridge  | Station: GHCND:USW00014806  | Feature: TMIN  | Station Count: 50  | Query Count: 150
Project: California Ridge  | Station: GHCND:USC00118740  | Feature: TAVG  | Station Count: 51  | Query Count: 151
Project: California Ridge  | Station: GHCND:USC00118740  | Feature: TMAX  | Station Count: 51  | Query Count: 152
Project: California Ridge  | Station: GHCND:USC00118740  | Feature: TMIN  | Station Count: 51  | Query Count: 153
Project: Miami  | Station: GHCND:USW00003024  | Feature: TAVG  | Station Count: 52  | Query Count: 154
Project: Miami  | Station: GHCND:USW00003024  | Feature: TMAX  | Station Count: 52  | Query Count: 

Project: Seiling  | Station: GHCND:USW00003932  | Feature: TMAX  | Station Count: 75  | Query Count: 224
Project: Seiling  | Station: GHCND:USW00003932  | Feature: TMIN  | Station Count: 75  | Query Count: 225
Project: Gratiot County  | Station: GHCND:USW00014845  | Feature: TAVG  | Station Count: 76  | Query Count: 226
Project: Gratiot County  | Station: GHCND:USW00014845  | Feature: TMAX  | Station Count: 76  | Query Count: 227
Project: Gratiot County  | Station: GHCND:USW00014845  | Feature: TMIN  | Station Count: 76  | Query Count: 228
Project: Gratiot County  | Station: GHCND:USW00014828  | Feature: TAVG  | Station Count: 77  | Query Count: 229
Project: Gratiot County  | Station: GHCND:USW00014828  | Feature: TMAX  | Station Count: 77  | Query Count: 230
Project: Gratiot County  | Station: GHCND:USW00014828  | Feature: TMIN  | Station Count: 77  | Query Count: 231
Project: Gratiot County  | Station: GHCND:USW00014836  | Feature: TAVG  | Station Count: 78  | Query Count: 232
Projec

Project: Victory(Carroll)  | Station: GHCND:USC00133632  | Feature: TMAX  | Station Count: 100  | Query Count: 299
Project: Victory(Carroll)  | Station: GHCND:USC00133632  | Feature: TMIN  | Station Count: 100  | Query Count: 300
Project: Victory(Carroll)  | Station: GHCND:USC00131277  | Feature: TAVG  | Station Count: 101  | Query Count: 301
Project: Victory(Carroll)  | Station: GHCND:USC00131277  | Feature: TMAX  | Station Count: 101  | Query Count: 302
Project: Victory(Carroll)  | Station: GHCND:USC00131277  | Feature: TMIN  | Station Count: 101  | Query Count: 303
Project: Victory(Carroll)  | Station: GHCND:USC00131442  | Feature: TAVG  | Station Count: 102  | Query Count: 304
Project: Victory(Carroll)  | Station: GHCND:USC00131442  | Feature: TMAX  | Station Count: 102  | Query Count: 305
Project: Victory(Carroll)  | Station: GHCND:USC00131442  | Feature: TMIN  | Station Count: 102  | Query Count: 306
Project: Bishop Hill I  | Station: GHCND:USW00014923  | Feature: TAVG  | Station

...

Project: Wiota Wind  | Station: GHCND:USC00137613  | Feature: TAVG  | Station Count: 1155  | Query Count: 3463
Project: Wiota Wind  | Station: GHCND:USC00137613  | Feature: TMAX  | Station Count: 1155  | Query Count: 3464
Project: Wiota Wind  | Station: GHCND:USC00137613  | Feature: TMIN  | Station Count: 1155  | Query Count: 3465
Project: AG Land 3  | Station: GHCND:USW00094989  | Feature: TAVG  | Station Count: 1156  | Query Count: 3466
Project: AG Land 3  | Station: GHCND:USW00094989  | Feature: TMAX  | Station Count: 1156  | Query Count: 3467
Project: AG Land 3  | Station: GHCND:USW00094989  | Feature: TMIN  | Station Count: 1156  | Query Count: 3468
Project: AG Land 3  | Station: GHCND:USC00130200  | Feature: TAVG  | Station Count: 1157  | Query Count: 3469
Project: AG Land 3  | Station: GHCND:USC00130200  | Feature: TMAX  | Station Count: 1157  | Query Count: 3470
Project: AG Land 3  | Station: GHCND:USC00130200  | Feature: TMIN  | Station Count: 1157  | Query Count: 3471
Project

Project: Ipswich  | Station: GHCND:USC00194502  | Feature: TAVG  | Station Count: 1179  | Query Count: 3535
Project: Ipswich  | Station: GHCND:USC00194502  | Feature: TMAX  | Station Count: 1179  | Query Count: 3536
Project: Ipswich  | Station: GHCND:USC00194502  | Feature: TMIN  | Station Count: 1179  | Query Count: 3537
Project: NREL - GE  | Station: GHCND:USC00058995  | Feature: TAVG  | Station Count: 1180  | Query Count: 3538
Project: NREL - GE  | Station: GHCND:USC00058995  | Feature: TMAX  | Station Count: 1180  | Query Count: 3539
Project: NREL - GE  | Station: GHCND:USC00058995  | Feature: TMIN  | Station Count: 1180  | Query Count: 3540
Project: NREL - GE  | Station: GHCND:USW00023062  | Feature: TAVG  | Station Count: 1181  | Query Count: 3541
Project: NREL - GE  | Station: GHCND:USW00023062  | Feature: TMAX  | Station Count: 1181  | Query Count: 3542
Project: NREL - GE  | Station: GHCND:USW00023062  | Feature: TMIN  | Station Count: 1181  | Query Count: 3543
Project: NREL - 

Project: Thousand Springs  | Station: GHCND:USW00004110  | Feature: TMAX  | Station Count: 1202  | Query Count: 3605
Project: Thousand Springs  | Station: GHCND:USW00004110  | Feature: TMIN  | Station Count: 1202  | Query Count: 3606
Project: Thousand Springs  | Station: GHCND:USW00094178  | Feature: TAVG  | Station Count: 1203  | Query Count: 3607
Project: Thousand Springs  | Station: GHCND:USW00094178  | Feature: TMAX  | Station Count: 1203  | Query Count: 3608
Project: Thousand Springs  | Station: GHCND:USW00094178  | Feature: TMIN  | Station Count: 1203  | Query Count: 3609
Project: Jiminy Peak Ski Resort  | Station: GHCND:USW00014763  | Feature: TAVG  | Station Count: 1204  | Query Count: 3610
Project: Jiminy Peak Ski Resort  | Station: GHCND:USW00014763  | Feature: TMAX  | Station Count: 1204  | Query Count: 3611
Project: Jiminy Peak Ski Resort  | Station: GHCND:USW00014763  | Feature: TMIN  | Station Count: 1204  | Query Count: 3612
Project: Jiminy Peak Ski Resort  | Station: GH

Project: Golden Acorn Casino  | Station: GHCND:USW00003178  | Feature: TAVG  | Station Count: 1226  | Query Count: 3676
Project: Golden Acorn Casino  | Station: GHCND:USW00003178  | Feature: TMAX  | Station Count: 1226  | Query Count: 3677
Project: Golden Acorn Casino  | Station: GHCND:USW00003178  | Feature: TMIN  | Station Count: 1226  | Query Count: 3678
Project: Golden Acorn Casino  | Station: GHCND:USW00053120  | Feature: TAVG  | Station Count: 1227  | Query Count: 3679
Project: Golden Acorn Casino  | Station: GHCND:USW00053120  | Feature: TMAX  | Station Count: 1227  | Query Count: 3680
Project: Golden Acorn Casino  | Station: GHCND:USW00053120  | Feature: TMIN  | Station Count: 1227  | Query Count: 3681
Project: Spearville I  | Station: GHCND:USW00013985  | Feature: TAVG  | Station Count: 1228  | Query Count: 3682
Project: Spearville I  | Station: GHCND:USW00013985  | Feature: TMAX  | Station Count: 1228  | Query Count: 3683
Project: Spearville I  | Station: GHCND:USW00013985  |

Project: Sky Volt  | Station: GHCND:USW00094991  | Feature: TMIN  | Station Count: 1248  | Query Count: 3744


In [29]:
datasetid='GHCND'
datatypeid="TAVG"
stationid='GHCND:USW00013969'
units='standard'
startdate='2016-01-01'
enddate='2016-12-31'
limit = 366

In [30]:
t = get_temp_data_daily_summary(stationid, datatypeid, units = 'standard', startdate = '2016-01-01', enddate='2016-12-31', limit =366, offset = 25)

In [31]:
t

,attributes,datatype,date,station,value
0,"H,,S,",TAVG,2016-01-01T00:00:00,GHCND:USW00013969,33.0
1,"H,,S,",TAVG,2016-01-02T00:00:00,GHCND:USW00013969,33.0
2,"H,,S,",TAVG,2016-01-03T00:00:00,GHCND:USW00013969,32.0
3,"H,,S,",TAVG,2016-01-04T00:00:00,GHCND:USW00013969,27.0
4,"H,,S,",TAVG,2016-01-05T00:00:00,GHCND:USW00013969,29.0
5,"H,,S,",TAVG,2016-01-06T00:00:00,GHCND:USW00013969,38.0
6,"H,,S,",TAVG,2016-01-07T00:00:00,GHCND:USW00013969,44.0
7,"H,,S,",TAVG,2016-01-08T00:00:00,GHCND:USW00013969,41.0
8,"H,,S,",TAVG,2016-01-09T00:00:00,GHCND:USW00013969,34.0
9,"H,,S,",TAVG,2016-01-11T00:00:00,GHCND:USW00013969,37.0


In [38]:
p = get_temp_data_daily_summary(stationid, datatypeid, units = 'standard', startdate = '2015-01-01', enddate='2015-12-31', limit =366, offset = 25)

In [39]:
p

,attributes,datatype,date,station,value
0,"H,,S,",TAVG,2015-01-01T00:00:00,GHCND:USW00013969,25.0
1,"H,,S,",TAVG,2015-01-02T00:00:00,GHCND:USW00013969,32.0
2,"H,,S,",TAVG,2015-01-03T00:00:00,GHCND:USW00013969,36.0
3,"H,,S,",TAVG,2015-01-04T00:00:00,GHCND:USW00013969,23.0
4,"H,,S,",TAVG,2015-01-05T00:00:00,GHCND:USW00013969,22.0
5,"H,,S,",TAVG,2015-01-06T00:00:00,GHCND:USW00013969,32.0
6,"H,,S,",TAVG,2015-01-07T00:00:00,GHCND:USW00013969,22.0
7,"H,,S,",TAVG,2015-01-08T00:00:00,GHCND:USW00013969,18.0
8,"H,,S,",TAVG,2015-01-09T00:00:00,GHCND:USW00013969,28.0
9,"H,,S,",TAVG,2015-01-10T00:00:00,GHCND:USW00013969,24.0


In [33]:
for i in temp_data_types:
    print (i)

TAVG
TMAX
TMIN
